In [1]:
#| default_exp fmf_biobert

# Found My Fitness Example
>"Here we'll try using elements from the biobertApi module to label biomedical terms in the comments section of a Found My Fitness Youtube video"

In [2]:
#|hide
# import biobert-bern 
from biobert_bern.biobertApi import *

ModuleNotFoundError: No module named 'biobert_bern'

In [ ]:
# !pip install pathlib

In [ ]:
import requests
import re
import pandas as pd
from nbdev.showdoc import *
import fastai
from pathlib import *
import numpy as np     

In [ ]:
path = Path('../..')
comment_csv= 'found_my_fitness_UCWF8SqJVNlx-ctXbLswcTcA_youtube_comments_only_122rows.csv'

In [ ]:
coms = pd.read_csv(path/comment_csv)

In [ ]:
coms.rename(columns = {'comment': 'comments'},inplace=True)

In [ ]:
coms.tail()

# Create Text Block
> Here we'll put all the text together

In [ ]:
# coms = coms[['textDisplay']]
# coms.rename(columns={'textDisplay':'comments'},inplace=True)

In [ ]:
text_block = create_text_block(df=coms);# text_block

In [ ]:
text_block[:500]

### Get Comment Spans

In [ ]:
# dfi = get_comment_spans_textblock(text_block); dfi.tail(3)

In [ ]:
len(text_block)

### Create Section Labels

In [ ]:
coms['textLen'] = coms.comidx.apply(lambda row: len(row))
coms['cumTxtLen'] = coms.textLen.cumsum()
# coms['section'] = np.ceil(coms.cumTxtLen/5000) # round all values up 
# coms.section = coms.section.astype(int)

In [ ]:
coms

In [ ]:
c2=coms

In [ ]:
c3 = c2[c2.cumTxtLen >5000]
c2 = c2[c2.cumTxtLen <5000]
c2['section'] = 1
c3['cumTxtLen'] = c3.textLen.cumsum()
c3 = c3[c3.cumTxtLen <5000]
c3['section'] = 2

## Now start splitting off all the cumsums <5000 and stack them back together.

In [ ]:
c2

In [ ]:
c3

In [71]:
### coms['cumTxtLen'] = coms.textLen.cumsum()

In [45]:
for o in blocks.text_block:
    print(len(o))

4811
5352
5250
5368
3691


In [ ]:
# # Send it all to Bern2
# output = query_plain(text=text_block);  

## Try by comment

In [35]:
df = pd.DataFrame()
for o in blocks.text_block[:2]:
    print( len(o))
#     out = query_plain(text=o)
#     df = df.append({'out':out},ignore_index=True)
# df.tail()

4811
5352


In [33]:
df.out[0]['annotations']

[{'id': ['CHEBI:16236'],
  'is_neural_normalized': False,
  'mention': 'alcohol',
  'obj': 'drug',
  'prob': 0.9939329028129578,
  'span': {'begin': 44, 'end': 51}},
 {'id': ['CUI-less'],
  'is_neural_normalized': False,
  'mention': 'rhonda',
  'obj': 'species',
  'prob': 0.3655291199684143,
  'span': {'begin': 333, 'end': 339}},
 {'id': ['CUI-less'],
  'is_neural_normalized': False,
  'mention': 'ayahuasca',
  'obj': 'species',
  'prob': 0.6412533521652222,
  'span': {'begin': 408, 'end': 417}},
 {'id': ['CHEBI:8614'],
  'is_neural_normalized': False,
  'mention': 'psylocybin',
  'obj': 'drug',
  'prob': 0.9987693428993225,
  'span': {'begin': 421, 'end': 431}},
 {'id': ['CHEBI:16236'],
  'is_neural_normalized': False,
  'mention': 'alcohol',
  'obj': 'drug',
  'prob': 0.7819486260414124,
  'span': {'begin': 467, 'end': 474}},
 {'id': ['NCBITaxon:9606'],
  'is_neural_normalized': False,
  'mention': 'patients',
  'obj': 'species',
  'prob': 0.9936095476150513,
  'span': {'begin': 485

In [26]:
df2 = pd.DataFrame()
for p,o in zip(df.index, df.out):    
    dft = pd.DataFrame(o['annotations'])
    dft['comidx'] = p
    df2 = df2.append(dft)
df2

,comidx


In [25]:
df2

,comidx


In [24]:
pd.DataFrame(df['annotations'])

KeyError: 'annotations'